In [120]:
import pandas as pd
import datetime
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



In [121]:
train_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Train.csv')
riders_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Riders.csv')
test_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Test.csv')


In [122]:
def time_from_midnight_in_seconds(data, column):
    data[column] = pd.to_datetime(data[column])
    return (data[column] - pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')


def delta_time(dataset, higher_time, lower_time):
    return dataset[higher_time] - dataset[lower_time]


In [123]:
# converting time in seconds from midnight
train_data['Placement - Time'] = time_from_midnight_in_seconds(train_data, 'Placement - Time')
train_data['Confirmation - Time'] = time_from_midnight_in_seconds(train_data, 'Confirmation - Time')
train_data['Pickup - Time'] = time_from_midnight_in_seconds(train_data, 'Pickup - Time')
train_data['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(train_data, 'Arrival at Pickup - Time')
train_data['Arrival at Destination - Time'] = time_from_midnight_in_seconds(train_data, 'Arrival at Destination - Time')

# converting time in seconds from midnight for test data
test_data['Placement - Time'] = time_from_midnight_in_seconds(test_data, 'Placement - Time')
test_data['Confirmation - Time'] = time_from_midnight_in_seconds(test_data, 'Confirmation - Time')
test_data['Pickup - Time'] = time_from_midnight_in_seconds(test_data, 'Pickup - Time')
test_data['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(test_data, 'Arrival at Pickup - Time')


In [124]:
# calculating delta_time for train data
delta_confirm_place_train = delta_time(train_data, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_train = delta_time(train_data, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_train = delta_time(train_data, 'Pickup - Time', 'Arrival at Pickup - Time')
delta_arrival_pickup_train = delta_time(train_data, 'Arrival at Destination - Time', 'Pickup - Time')
delta_placement_arrival_train = delta_time(train_data, 'Arrival at Destination - Time', 'Placement - Time')

delta_confirm_place_test = delta_time(test_data, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_test = delta_time(test_data, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_test = delta_time(test_data, 'Pickup - Time', 'Arrival at Pickup - Time')


In [125]:
train_with_rider_info = train_data.merge(riders_data, on='Rider Id')
test_with_rider_info = test_data.merge(riders_data, on='Rider Id')


In [126]:
train_with_rider_info = train_with_rider_info[train_with_rider_info['Placement - Weekday (Mo = 1)'] == train_with_rider_info['Confirmation - Weekday (Mo = 1)']]
train_with_rider_info = train_with_rider_info[train_with_rider_info['Placement - Day of Month'] == train_with_rider_info['Confirmation - Day of Month']]


In [127]:
train_with_rider_info['Temperature'].fillna(train_with_rider_info['Temperature'].mean(), inplace=True)
test_with_rider_info['Temperature'].fillna(test_with_rider_info['Temperature'].mean(), inplace=True)


In [128]:
# label encoding of personal/business column for train data
labelencoder_personal_business = LabelEncoder()
train_with_rider_info['Personal or Business'] = labelencoder_personal_business.fit_transform(train_with_rider_info['Personal or Business'])

# label encoding of personal/business column for test data
labelencoder_personal_business = LabelEncoder()
test_with_rider_info['Personal or Business'] = labelencoder_personal_business.fit_transform(test_with_rider_info['Personal or Business'])


In [129]:
# one hot encoding of the train_data['Platform Type'] column
train_with_rider_info['Platform Type'] = train_with_rider_info['Platform Type'].astype('category')
train_with_rider_info = pd.concat([train_with_rider_info.drop(columns=['Platform Type']), pd.get_dummies(train_with_rider_info['Platform Type'])], axis=1)



In [130]:
# one hot encoding of the test_data['Platform Type'] column
test_with_rider_info['Platform Type'] = test_with_rider_info['Platform Type'].astype('category')
test_with_rider_info = pd.concat([test_with_rider_info.drop(columns=['Platform Type']), pd.get_dummies(test_with_rider_info['Platform Type'])], axis=1)


In [131]:
# drop redundant columns train data
train_with_rider_info.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                                    'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)',
                                    'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)',
                                    'Arrival at Destination - Day of Month',
                                    'Arrival at Destination - Weekday (Mo = 1)',
                                    'Vehicle Type', 'Order No',
                                    'User Id', 'Rider Id',
                                    'Precipitation in millimeters'], inplace=True)


In [96]:
# drop redundant columns test data
test_with_rider_info.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                                   'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)',
                                   'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)',
                                   'Vehicle Type', 'Order No',
                                   'User Id', 'Rider Id',
                                   'Precipitation in millimeters'], inplace=True)


In [132]:
train_with_rider_info.rename(columns={1: "Platform Type 1", 2: "Platform Type 2",
                                      3: "Platform Type 3", 4: "Platform Type 4"}, inplace=True)

test_with_rider_info.rename(columns={1: "Platform Type 1", 2: "Platform Type 2",
                                     3: "Platform Type 3", 4: "Platform Type 4"}, inplace=True)


In [133]:
train_with_rider_info.rename(columns={'Placement - Day of Month': 'Day of Month',
                                      'Placement - Weekday (Mo = 1)': 'Weekday (Mo = 1)'})

test_with_rider_info.rename(columns={'Placement - Day of Month': 'Day of Month',
                                     'Placement - Weekday (Mo = 1)': 'Weekday (Mo = 1)'})


,Order No,User Id,Vehicle Type,Personal or Business,Day of Month,Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,...,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Platform Type 1,Platform Type 2,Platform Type 3,Platform Type 4
0,Order_No_19248,User_Id_3355,Bike,0,27,3,60250.0,27,3,60269.0,...,36.822390,Rider_Id_192,439,1511,13.3,171,0,0,1,0
1,Order_No_2699,User_Id_592,Bike,1,26,2,53001.0,26,2,53236.0,...,36.760677,Rider_Id_192,439,1511,13.3,171,1,0,0,0
2,Order_No_21486,User_Id_478,Bike,0,29,5,42905.0,29,5,43092.0,...,36.790237,Rider_Id_192,439,1511,13.3,171,0,0,1,0
3,Order_No_19336,User_Id_2801,Bike,0,2,1,33801.0,2,1,33824.0,...,36.787118,Rider_Id_192,439,1511,13.3,171,0,0,1,0
4,Order_No_20374,User_Id_3465,Bike,0,14,4,46559.0,14,4,48164.0,...,36.782203,Rider_Id_192,439,1511,13.3,171,0,0,1,0
5,Order_No_9830,User_Id_1926,Bike,0,30,4,45557.0,30,4,45617.0,...,36.835960,Rider_Id_192,439,1511,13.3,171,0,0,1,0
6,Order_No_17505,User_Id_978,Bike,1,11,1,55593.0,11,1,55667.0,...,36.883519,Rider_Id_192,439,1511,13.3,171,1,0,0,0
7,Order_No_23802,User_Id_2445,Bike,1,17,4,46727.0,17,4,46735.0,...,36.806930,Rider_Id_192,439,1511,13.3,171,0,1,0,0
8,Order_No_12736,User_Id_3647,Bike,0,17,5,46655.0,17,5,46757.0,...,36.823907,Rider_Id_868,488,273,14.4,45,0,0,1,0
9,Order_No_6400,User_Id_299,Bike,0,3,1,38311.0,3,1,38326.0,...,36.767784,Rider_Id_868,488,273,14.4,45,0,0,1,0


In [134]:
X = train_with_rider_info.drop(columns='Time from Pickup to Arrival')
Y = train_with_rider_info['Time from Pickup to Arrival']


In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [164]:
regressor = RandomForestRegressor(n_estimators=160, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [138]:
mean_squared_error(y_test/60, y_pred/60)


98.84753488312491

In [168]:
grid_param = {
    'n_estimators': [150,160,170,180]
}


In [169]:
gd_sr = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=grid_param,
    scoring='neg_mean_absolute_error',
    cv=4,
    n_jobs=-1)


In [170]:
gd_sr.fit(X_train, y_train)


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [150, 160, 170, 180]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [167]:
best_parameters = gd_sr.best_params_


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'